# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
# import libraries
import re
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
import seaborn as sns
from IPython import display

%matplotlib inline


from sqlalchemy import create_engine
from collections import Counter


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neural_network import MLPClassifier


import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# load data from database

engine = create_engine('sqlite:///DisasterResponse.db')

df = pd.read_sql_table('disaster_messages', engine)
X = df['message']
Y = df.loc[:, 'related':'direct_report']

In [5]:
print('Xhead:{} and Y:head{}'.format(X.head(), Y.head()))

Xhead:0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object and Y:head   related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone      ...        \
0                  0         0         0            0      ...         
1                  0         0         0            0      ...         
2            

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    # for this function we will be using the same cleaning steps as used in clean_tokenzie exercise during the NLP pipeline lesson
    #as learnt in building NLP pipeline process, we will clean data first by first replacing anything that is not a digit or an alphabet with space
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Now that the text has been cleaned i.e. punctuations have been removed we can tokenize
    tokens = word_tokenize(text)
    #after using tokenization, we can initiate lemmatizer
    lemmatizer =  WordNetLemmatizer()
    #Now we can initiate Stemming
    stemmer = PorterStemmer()
    clean_tokens = []
    for tok in tokens:
        #we will now lemmatize token using verb, and make it lower case and remove empty space
        clean_tok = lemmatizer.lemmatize(tok, pos='v').lower().strip()
        #now we can stem token by using stemmer
        clean_tok = stemmer.stem(clean_tok)
        clean_tokens.append(clean_tok)

    return clean_tokens

In [7]:
# Now we can test our function to see if it is actually working

text = 'Warren Deutrom, the chief executive of Cricket Ireland, has reiterated his interest in Ireland touring Pakistan after being in Lahore for the recent T20Is against Sri Lanka and being left impressed with the way the series went off.'


tokenize(text)

print("excellent!, our function works")

excellent!, our function works


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
])

In [9]:
X.shape

(26216,)

In [10]:
Y.shape

(26216, 36)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# train classifier
pipeline.fit(X_train, Y_train)

# predict on test data
Y_predict = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
cat_names = Y_test.columns.tolist()
df_y_predict = pd.DataFrame(Y_predict, columns = cat_names)
df_y_predict.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
len(cat_names)

36

In [16]:
#Now we try to print all scores, precision, recall, and f-1score to see the performance of the mode. What we will do is to print the score for reach category

for i in range(len(cat_names)):
    print(cat_names[i],\
          '\n',\
          classification_report(Y_test.iloc[:,i], df_y_predict.iloc[:,i]))

related 
              precision    recall  f1-score   support

          0       0.61      0.22      0.33      1266
          1       0.79      0.95      0.86      3938
          2       0.29      0.12      0.18        40

avg / total       0.74      0.77      0.73      5244

request 
              precision    recall  f1-score   support

          0       0.91      0.97      0.94      4349
          1       0.77      0.51      0.61       895

avg / total       0.88      0.89      0.88      5244

offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.14      0.04      0.06        26

avg / total       0.99      0.99      0.99      5244

aid_related 
              precision    recall  f1-score   support

          0       0.76      0.86      0.81      3113
          1       0.75      0.59      0.66      2131

avg / total       0.75      0.75      0.75      5244

medical_help 
              precision    reca

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
#Now we get parameters 

pipeline.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fac6e898510>, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=42),
              n_jobs=1))],
 'tfidf': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), no

In [18]:
# parameters that we should optimize 'tfidf__max_df': 1.0,
#  'tfidf__max_features': None,
#  'tfidf__min_df': 1,
#  'tfidf__ngram_range': (1, 1),
# 'clf__estimator__learning_rate': 1.0,
#  'clf__estimator__n_estimators': 50,
parameters = {'tfidf__ngram_range': ((1, 1), (1,3))}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 38.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...timator=None,
          learning_rate=1.0, n_estimators=50, random_state=42),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__ngram_range': ((1, 1), (1, 3))},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [22]:
Y_predict2 = cv.predict(X_test)
df_y_pred2 = pd.DataFrame(Y_predict2, columns =cat_names )
df_y_pred2.head()



,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
for i in range(len(cat_names)):
    print(cat_names[i],\
          '\n',\
          classification_report(Y_test.iloc[:,i], df_y_pred2.iloc[:,i]))

related 
              precision    recall  f1-score   support

          0       0.61      0.22      0.33      1266
          1       0.79      0.95      0.86      3938
          2       0.29      0.12      0.18        40

avg / total       0.74      0.77      0.73      5244

request 
              precision    recall  f1-score   support

          0       0.91      0.97      0.94      4349
          1       0.77      0.51      0.61       895

avg / total       0.88      0.89      0.88      5244

offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.14      0.04      0.06        26

avg / total       0.99      0.99      0.99      5244

aid_related 
              precision    recall  f1-score   support

          0       0.76      0.86      0.81      3113
          1       0.75      0.59      0.66      2131

avg / total       0.75      0.75      0.75      5244

medical_help 
              precision    reca

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [27]:
filename = 'classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.